In [154]:
import numpy as np
import pandas as pd
import math

from sklearn.datasets import make_classification
from sklearn.datasets import make_regression

from sklearn.metrics import accuracy_score
from sklearn.metrics import r2_score

from supervised.decision_tree import DecisionTreeClassifier, DecisionTreeRegression


In [169]:
class RandomForest:
    """    Random forest common class.    """
    def __init__(self, trees, n_trees, max_feature ,
                 prediction_aggrigation_calculation):
        """
        :param trees: List - list of tree objects. classification tree/regression trees.
        :param n_trees: int - How may estimators/tree should be used for random forest building.
        :param max_feature: Int - How many features can be used for a tree from the whole features.
        :param prediction_aggrigation_calculation: Function - Aggication function to find the prediction.
        """
        self.n_estimators = n_trees
        self.max_features = max_feature
        self.tree_feature_indexes = []
        self.prediction_aggrigation_calculation = prediction_aggrigation_calculation 
        # Initialize the trees.
        self.trees = trees

    def _make_random_suset(self, X, y, n_subsets, replasment=True):
        """
        Creata a random subset of dataset with/without replacement.

        :param X: Depentand variables.
        :param y: Indepentant variable.
        :param n_subsets: Number of subset we need.
        :param replasment: Boolena - Can we use the data sample again or not.
        """
        subset = []
        # use 100% of data when replacement is true , use 50% otherwise.
        sample_size = (X.shape[0] if replasment else (X.shape[0] // 2))

        # First concadinate the X and y datasets in order to make a choice.
        Xy = np.concatenate((X, y), axis=1)
        np.random.shuffle(Xy)
        # Select randome subset of data with replacement.
        for i in range(n_subsets):
            index = np.random.choice(range(sample_size), size=np.shape(range(sample_size)), replace=replasment)
            X = Xy[index][:, :-1]
            y = Xy[index][: , -1]
            subset.append({"X" : X, "y": y})
        return subset

    def train(self, X, y):
        """
        Build the model.
        :param X: Depentand variables.
        :param y: Indepentant variable.
        """
        # if the max_features is not given then select it as square root of no on feature availabe.
        n_features = X.shape[1]
        if self.max_features == None:
            self.max_features = int(math.sqrt(n_features))

        # Split the dataset into number of subsets equal to n_estimators.
        subsets = self._make_random_suset(X, y, self.n_estimators)

        for i, subset in enumerate(subsets):
            X_subset , y_subset = subset["X"], subset["y"]
            # select a random sucset of features for each tree. This is called feature bagging.
            idx = np.random.choice(range(n_features), size=self.max_features, replace=True)
            # track this for prediction.
            self.tree_feature_indexes.append(idx)
            # Get the X with the selected features only.
            X_subset = X_subset[:, idx]

            # change the y_subet to i dimentional array.
            y_subset = np.expand_dims(y_subset, axis =1)
            # build the model with selected features and selected random subset from dataset.
            self.trees[i].train(X_subset, y_subset)

    def predict(self, test_X):
        """
        Predict the new samples.

        :param test_X: Depentant variables for prediction.
        """
        # predict each sample one by one.
        y_preds = np.empty((test_X.shape[0], self.n_estimators))
        # find the prediction from each tree for eeach samples
        for i, tree in enumerate(self.trees):
            features_index = self.tree_feature_indexes[i]
            X_selected_features = test_X[:, features_index]
            if isinstance(tree, DecisionTreeClassifier):
                y_preds[:, i] = tree.predict(X_selected_features).reshape((-1,))
            else:
                y_preds[:, i] = tree.predict(X_selected_features)
        # find the arrgrecated output.
        y_pred = self.prediction_aggrigation_calculation(y_preds)

        return y_pred



# Random forest Classifier from scratch

In [170]:
class RandomForestClassifier(RandomForest):
    """Rnadom forest for classification task."""
    def __init__(self, max_feature, max_depth, n_trees=100, min_sample_split=2, min_impurity=1e-7):
        """
        :param max_depth: Int - Max depth of each tree.
        :param n_trees: Int - Number of trees/estimetors.
        :param min_sample_split: Int - minimum samples for a node to have before going for split.
        :param min_impurity: Int - Min inpurity a node can have.
        """
        self.prediction_aggrigation_calculation = self._maximum_vote_calculation
        
        # Initializing the trees.
        self.trees = []
        for _ in range(n_trees):
            self.trees.append(DecisionTreeClassifier(min_sample_split=min_sample_split,
                                                     min_impurity=min_impurity, max_depth=max_depth))

        super().__init__(trees=self.trees, n_trees=n_trees,max_feature=max_feature,
                         prediction_aggrigation_calculation=self.prediction_aggrigation_calculation)
    
    def _maximum_vote_calculation(self, y_preds):
        """
        Find which prediction class has higest frequency in all tree prediction for each sampple.

        :param y_preds: Prediction value from number of estimators trees.
        """
        # create a empty array to store the prediction.
        y_pred = np.empty((y_preds.shape[0], 1))
        # iterate over all the data samples.
        for i, sample_predictions in enumerate(y_preds):
            y_pred[i] = np.bincount(sample_predictions.astype('int')).argmax()

        return y_pred

In [171]:
# X = np.array([["Green", 3], ["yello", 3], ["orange_color", 2], ["orange_color", 2], ["red", 1]])
# y = np.array(["apply", "apply", "orange", "orange", "cherry"])
# X = pd.DataFrame(X)
# y = pd.DataFrame(y)
# y.head

# Define the traning data.
X, y = make_classification(n_samples=1000, n_classes=2, n_features=5)

# Chnage the shape of the target to 1 dimentional array.
y = y[:, np.newaxis]

print("="*100)
print("Number of training data samples-----> {}".format(X.shape[0]))
print("Number of training features --------> {}".format(X.shape[1]))
print("Shape of the target value ----------> {}".format(y.shape))

Number of training data samples-----> 1000
Number of training features --------> 5
Shape of the target value ----------> (1000, 1)


In [172]:
# display the data.
data = pd.DataFrame(X)
data.head()

,0,1,2,3,4
0,1.215307,-0.660581,-1.350245,-0.231831,-0.398433
1,-1.415503,-1.595183,-1.155783,0.973351,-0.314888
2,-1.775128,-1.906776,-1.341324,1.192777,1.737915
3,-0.373321,-0.699046,-0.625991,0.339499,0.711422
4,0.603384,0.385183,0.152516,-0.327224,-1.461580


In [173]:
# display the data.
data_y = pd.DataFrame(y)
data_y.head()

,0
0,1
1,0
2,0
3,0
4,1


In [174]:
#define the parameters

param = {
    "n_neibours" : 5
}
print("="*100)
random_forest_cla = RandomForestClassifier(n_trees=3, max_feature=2, min_sample_split=2, max_depth=45)

# Train the model.
random_forest_cla.train(X, y) 

# Predict the values.
y_pred = random_forest_cla.predict(X)

#calculate accuracy.
acc = np.sum(y==y_pred)/X.shape[0]
print("="*100)
print("Accuracy of the prediction is {}".format(acc))

Accuracy of the prediction is 0.965


# Random forest Regression from scratch

In [175]:
class RandomForestRegression(RandomForest):
    """Rnadom forest for classification task."""
    def __init__(self, max_feature, max_depth, n_trees=100, min_sample_split=2, min_impurity=1e-7):
        """
        :param max_depth: Int - Max depth of each tree.
        :param n_trees: Int - Number of trees/estimetors.
        :param min_sample_split: Int - minimum samples for a node to have before going for split.
        :param min_impurity: Int - Min inpurity a node can have.
        """
        self.prediction_aggrigation_calculation = self._mean_calculation
        
        # Initializing the trees.
        self.trees = []
        for _ in range(n_trees):
            self.trees.append(DecisionTreeRegression(min_sample_split=min_sample_split,
                                                     min_impurity=min_impurity, max_depth=max_depth))

        super().__init__(trees=self.trees, n_trees=n_trees,max_feature=max_feature,
                         prediction_aggrigation_calculation=self.prediction_aggrigation_calculation)
    
    def _mean_calculation(self, y_preds):
        """
        Find mean prediction of all tree prediction for each sampple.

        :param y_preds: Prediction value from number of estimators trees.
        """
        # create a empty array to store the prediction.
        y_pred = np.empty((y_preds.shape[0], 1))
        # iterate over all the data samples.
        for i, sample_predictions in enumerate(y_preds):
            y_pred[i] = np.mean(sample_predictions)

        return y_pred

In [176]:
# Define the traning data.
X, y = make_regression(n_samples=1000, n_features=8)

# Chnage the shape of the target to 1 dimentional array.
y = y[:, np.newaxis]

print("="*100)
print("Number of training data samples-----> {}".format(X.shape[0]))
print("Number of training features --------> {}".format(X.shape[1]))
print("Shape of the target value ----------> {}".format(y.shape))

Number of training data samples-----> 1000
Number of training features --------> 8
Shape of the target value ----------> (1000, 1)


In [177]:
# display the data.
data_y = pd.DataFrame(y)
data_y.head()

,0
0,261.258202
1,0.393038
2,32.156202
3,128.667816
4,-1.196097


In [178]:
#define the parameters
print("="*100)
random_forest_reg = RandomForestRegression(n_trees=3, max_feature=3, min_sample_split=2, max_depth=45)

# Train the model.
random_forest_reg.train(X, y) 

# Predict the values.
y_pred = random_forest_reg.predict(X)
#Root mean square error.
score = r2_score(y, y_pred)
print("The r2_score of the trained model", score)

The r2_score of the trained model 0.7698232305798236


Decision Tree (scratch coding) - The much know Explanation :)

**Decision Trees** -->
 To put in simple words , It is a bunch of  if-else statements ordered in a binary tree format. Building the tree would start from the root node with whole training dataset. Each node either splitted into 2 branches namely true branch and false branch or will remain as a leaf node(under some condition which is when min_sample_split reached or when the node is pure). 
The split will happen based on a yes or no question(if-else statement) -->    for a single data sample if the answer is true then it will go under true branch else vice versa. 
That is pretty much about the over all process......😃 

**Here what we need next is:**
1. How can we measure a node is pure/not?
2. How to choose/find a best question to split the data?
Both the question have the same answer --> Based on the  **Impurity function **

- For **Classification** the impurity function is ***Information Gain (IG)***
- For **Regression** the impurity function is ***Variance reduction***

**Answer:**
1. If the impurity function is **Zero** then the node is pure (i.e. it has data from one single class on classification data)
2. Select a question which has the highest impurity value.

Let's learn about Impurity function -------

- **Information Gain**

Value will range from 0-1  ---> 0 means 100% pure --- 1 means 100% not pure. 

```IG = Entropy(y) - p * Entropy(true_branch_y) - (1 - p) * Entropy(false_branch_y) ```
Here , 
p --> probability of true branch data. [ len(true_branch_y) / len(y)]

`Entropy = - p1 * (log(p1) / log(2)) - p2 * (log(p2) / log(2)) - p3 * (log(p3) / log(3)) ....`

p1, p2, p3 --> Probability of each class.

- **Variance reduction**

Value will range from 0-1.

```VR = Variance(y) - (fraction * variance(true_branch_y) + (1 - fraction) * variance(false_branch_y))```
Here , 
fraction --> probability of true branch data. [ len(true_branch_y) / len(y)]

**Steps in summary:** 😀
1. Start from the root node with whole training data.
2. Find a best question to split the data for that node.
    2.1 . iterate over all the features .
    2.2 . Find the unique values in the branch. Iterate over all unique values.
    2.3 . For each (feature , unique_value) --> this frames a question and find impurity funtion for this question.
    2.4 . Find Impurity function values for all the (feature , unique_value) pair.
    2.5 . Choose a pair which has highest impurity function value.
3. Split the node's data into true branch and false branch based on this question.
4. Repeat 2 & 3 .
5. stop when min_sample_split(minimum data a node should have before the split) reached or impurity of a node has became zero.

**Prediction**:

Classification - traverse a data sample from the top of the tree to the bottom and assign the class which has maximum frequency in that leaf node as a prediction. 

Regression -  traverse a data sample from top of the tree to the bottom and assign the mean of target values in that leaf node as a prediction.

why we should use Decision Trees:
- Easy to build and interpret.
- Missing values and outlier have less significance on prediction.
- Training 